In [26]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random as pyrandom
from numpy import random
from sklearn import svm
from __future__ import print_function
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from random import choice
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import validation_curve
from pylab import *  
from matplotlib.ticker import MultipleLocator, FormatStrFormatter  
from collections import Counter
from sklearn.calibration import CalibratedClassifierCV
%matplotlib inline

In [2]:
# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 9.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
train_data=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/train_data.npy")
train_label=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/train_label.npy")
test_data=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/test_data.npy")
test_label=np.load("D:/GitHub/Neural-Network-Theory-and-Applications-Homework-SJTU2018/Homework Assignment 2/data_hw2/test_label.npy")

##原来的标签是{-1,0,1}，现在变为三堆样本标签，正样本是0，负样本是1

In [4]:
train_label_0=np.where(train_label == 0, 0, 1)
train_label_1=np.where(train_label == 1, 0, 1)
train_label_negative1=np.where(train_label == -1, 0, 1)

test_label_0=np.where(test_label == 0, 0, 1)
test_label_1=np.where(test_label == 1, 0, 1)
test_label_negative1=np.where(test_label == -1, 0, 1)

##将三个二分类问题的索引随机打乱,二分类问题随机分为8组，共24组

In [28]:
range_0=list(range(37367))
pyrandom.shuffle(range_0)
range_1=list(range(37367))
pyrandom.shuffle(range_1)
range_negative1=list(range(37367))
pyrandom.shuffle(range_negative1)

In [34]:
int(8*0.125*37366)

37366

现在的目的是，将打乱的range_x分成8组,

In [50]:
range_0_=[8,]
range_1_=[8,]
range_negative1_=[8,]
for i in range(8):
    range_0_[i].append(range_0[int(i*0.125*37366):int((i+1)*0.125*37366)])
for i in range(8):
    range_1_[i].append(range_1[int(i*0.125*37366):int((i+1)*0.125*37366)])
for i in range(8):
    range_negative1_[i].append(range_negative1[int(i*0.125*37366):int((i+1)*0.125*37366)])

AttributeError: 'int' object has no attribute 'append'

##为每个二分类问题建立8个分类器，一共24个SVM分类器

In [ ]:
svc_0_11=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_12=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_13=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_14=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_21=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_22=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_23=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_0_24=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 

svc_1_11=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_12=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_13=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_14=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_21=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_22=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_23=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 
svc_1_24=CalibratedClassifierCV(svm.LinearSVC(C=2e-7)) 

svc_negative1_11=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_12=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_13=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_14=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_21=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_22=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_23=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 
svc_negative1_24=CalibratedClassifierCV(svm.LinearSVC(C=1e-6)) 

##每个二分类问题训练8个分类器，一共训练24个分类器

In [ ]:
svc_0_11.fit(train_data[train_index_0_11], train_label_0[train_index_0_11])
svc_0_12.fit(train_data[train_index_0_12], train_label_0[train_index_0_12])
svc_0_13.fit(train_data[train_index_0_13], train_label_0[train_index_0_13])
svc_0_14.fit(train_data[train_index_0_14], train_label_0[train_index_0_14])
svc_0_21.fit(train_data[train_index_0_21], train_label_0[train_index_0_21])
svc_0_22.fit(train_data[train_index_0_22], train_label_0[train_index_0_22])
svc_0_23.fit(train_data[train_index_0_23], train_label_0[train_index_0_23])
svc_0_24.fit(train_data[train_index_0_24], train_label_0[train_index_0_24])

In [ ]:
svc_1_11.fit(train_data[train_index_1_11], train_label_1[train_index_1_11])
svc_1_12.fit(train_data[train_index_1_12], train_label_1[train_index_1_12])
svc_1_13.fit(train_data[train_index_1_13], train_label_1[train_index_1_13])
svc_1_14.fit(train_data[train_index_1_14], train_label_1[train_index_1_14])
svc_1_21.fit(train_data[train_index_1_21], train_label_1[train_index_1_21])
svc_1_22.fit(train_data[train_index_1_22], train_label_1[train_index_1_22])
svc_1_23.fit(train_data[train_index_1_23], train_label_1[train_index_1_23])
svc_1_24.fit(train_data[train_index_1_24], train_label_1[train_index_1_24])

In [ ]:
svc_negative1_11.fit(train_data[train_index_negative1_11], train_label_negative1[train_index_negative1_11])
svc_negative1_12.fit(train_data[train_index_negative1_12], train_label_negative1[train_index_negative1_12])
svc_negative1_13.fit(train_data[train_index_negative1_13], train_label_negative1[train_index_negative1_13])
svc_negative1_14.fit(train_data[train_index_negative1_14], train_label_negative1[train_index_negative1_14])
svc_negative1_21.fit(train_data[train_index_negative1_21], train_label_negative1[train_index_negative1_21])
svc_negative1_22.fit(train_data[train_index_negative1_22], train_label_negative1[train_index_negative1_22])
svc_negative1_23.fit(train_data[train_index_negative1_23], train_label_negative1[train_index_negative1_23])
svc_negative1_24.fit(train_data[train_index_negative1_24], train_label_negative1[train_index_negative1_24])

##实现min-max-modular

In [ ]:
test_pred_0_11 = svc_0_11.predict_proba(test_data)[:,0]
test_pred_0_12 = svc_0_12.predict_proba(test_data)[:,0]
test_pred_0_13 = svc_0_13.predict_proba(test_data)[:,0]
test_pred_0_14 = svc_0_14.predict_proba(test_data)[:,0]
test_pred_0_21 = svc_0_21.predict_proba(test_data)[:,0]
test_pred_0_22 = svc_0_22.predict_proba(test_data)[:,0]
test_pred_0_23 = svc_0_23.predict_proba(test_data)[:,0]
test_pred_0_24 = svc_0_24.predict_proba(test_data)[:,0]

test_pred_1_11 = svc_1_11.predict_proba(test_data)[:,0]
test_pred_1_12 = svc_1_12.predict_proba(test_data)[:,0]
test_pred_1_13 = svc_1_13.predict_proba(test_data)[:,0]
test_pred_1_14 = svc_1_14.predict_proba(test_data)[:,0]
test_pred_1_21 = svc_1_21.predict_proba(test_data)[:,0]
test_pred_1_22 = svc_1_22.predict_proba(test_data)[:,0]
test_pred_1_23 = svc_1_23.predict_proba(test_data)[:,0]
test_pred_1_24 = svc_1_24.predict_proba(test_data)[:,0]

test_pred_negative1_11 = svc_negative1_11.predict_proba(test_data)[:,0]
test_pred_negative1_12 = svc_negative1_12.predict_proba(test_data)[:,0]
test_pred_negative1_13 = svc_negative1_13.predict_proba(test_data)[:,0]
test_pred_negative1_14 = svc_negative1_14.predict_proba(test_data)[:,0]
test_pred_negative1_21 = svc_negative1_21.predict_proba(test_data)[:,0]
test_pred_negative1_22 = svc_negative1_22.predict_proba(test_data)[:,0]
test_pred_negative1_23 = svc_negative1_23.predict_proba(test_data)[:,0]
test_pred_negative1_24 = svc_negative1_24.predict_proba(test_data)[:,0]

In [ ]:
test_pred_0_1x=np.vstack([test_pred_0_11,test_pred_0_12,test_pred_0_13,test_pred_0_14]) #test_pred_0_1x.shape=(4, 13588)
test_pred_0_2x=np.vstack([test_pred_0_21,test_pred_0_22,test_pred_0_23,test_pred_0_24])
test_pred_1_1x=np.vstack([test_pred_1_11,test_pred_1_12,test_pred_1_13,test_pred_1_14])
test_pred_1_2x=np.vstack([test_pred_1_21,test_pred_1_22,test_pred_1_23,test_pred_1_24])
test_pred_negative1_1x=np.vstack([test_pred_negative1_11,test_pred_negative1_12,test_pred_negative1_13,test_pred_negative1_14])
test_pred_negative1_2x=np.vstack([test_pred_negative1_21,test_pred_negative1_22,test_pred_negative1_23,test_pred_negative1_24])

In [ ]:
test_pred_0=[]
test_pred_1=[]
test_pred_negative1=[]
for i in range(13588):
    min_index_0_1x=int(np.where(test_pred_0_1x[:,i]==np.min(test_pred_0_1x[:,i]))[0])
    min_index_0_2x=int(np.where(test_pred_0_2x[:,i]==np.min(test_pred_0_2x[:,i]))[0])
    if test_pred_0_1x[min_index_0_1x,i] > test_pred_0_2x[min_index_0_2x,i]:
        test_pred_0.append(test_pred_0_1x[min_index_0_1x,i])
    else:
        test_pred_0.append(test_pred_0_2x[min_index_0_2x,i])
        
for i in range(13588):
    min_index_1_1x=int(np.where(test_pred_1_1x[:,i]==np.min(test_pred_1_1x[:,i]))[0])
    min_index_1_2x=int(np.where(test_pred_1_2x[:,i]==np.min(test_pred_1_2x[:,i]))[0])
    if test_pred_1_1x[min_index_1_1x,i] > test_pred_1_2x[min_index_1_2x,i]:
        test_pred_1.append(test_pred_1_1x[min_index_1_1x,i])
    else:
        test_pred_1.append(test_pred_1_2x[min_index_1_2x,i])
        
for i in range(13588):
    min_index_negative1_1x=int(np.where(test_pred_negative1_1x[:,i]==np.min(test_pred_negative1_1x[:,i]))[0])
    min_index_negative1_2x=int(np.where(test_pred_negative1_2x[:,i]==np.min(test_pred_negative1_2x[:,i]))[0])
    if test_pred_negative1_1x[min_index_negative1_1x,i] > test_pred_negative1_2x[min_index_negative1_2x,i]:
        test_pred_negative1.append(test_pred_negative1_1x[min_index_negative1_1x,i])
    else:
        test_pred_negative1.append(test_pred_negative1_2x[min_index_negative1_2x,i])

In [ ]:
test_pred=[]
train_pred=[]
for i in range(13588):
    maxone=max(test_pred_0[i],test_pred_1[i],test_pred_negative1[i])
    if maxone == test_pred_0[i]:
        test_pred.append(0)
    elif maxone == test_pred_1[i]:
        test_pred.append(1)
    elif maxone == test_pred_negative1[i]:
        test_pred.append(-1)

In [ ]:
test_label_roc=np.concatenate((test_label_0,test_label_1,test_label_negative1)) 
test_pred_roc=np.concatenate((test_pred_0,test_pred_1,test_pred_negative1))

In [ ]:
#画ROC曲线和计算AUC  
fpr_0, tpr_0, thresholds_0 = roc_curve(test_label_0, test_pred_0,pos_label=0)#
roc_auc_0 = auc(fpr_0, tpr_0)    
fpr_1, tpr_1, thresholds_1 = roc_curve(test_label_1, test_pred_1,pos_label=0)#
roc_auc_1 = auc(fpr_1, tpr_1)  
fpr_negative1, tpr_negative1, thresholds_negative1 = roc_curve(test_label_negative1, test_pred_negative1,pos_label=0)#
roc_auc_negative1 = auc(fpr_negative1, tpr_negative1) 
fpr_m, tpr_m, thresholds_m = roc_curve(test_label_roc, test_pred_roc,pos_label=0)
roc_auc_m = auc(fpr_m, tpr_m)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr_0, tpr_0, color='darkorange',
         lw=lw, label='class(0) ROC curve (area = %0.2f)' % roc_auc_0)
plt.plot(fpr_1, tpr_1, color='green',
         lw=lw, label='class(1) ROC curve (area = %0.2f)' % roc_auc_1)
plt.plot(fpr_negative1, tpr_negative1, color='blue',
         lw=lw, label='class(-1) ROC curve (area = %0.2f)' % roc_auc_negative1)
plt.plot(fpr_m, tpr_m, color='black',linestyle=':',
         lw=6, label='average ROC curve (area = %0.2f)' % roc_auc_m)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve of random-M3SVM')
plt.legend(loc="lower right")
plt.savefig('roc priori-m3svm.png')
plt.show()

In [ ]:
print(accuracy_score(test_label, test_pred))

In [ ]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(test_label, test_pred, target_names=target_names))